In [4]:
import requests
from bs4 import BeautifulSoup
import json
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [5]:
data = pd.read_csv('film_reviews_4493_original.csv',  sep='\t')

In [6]:
pd.set_option('display.max_rows',753)

In [7]:
#good_review = 'interesting fascinating inspirational'
#bad_review = 'boring disgusting uninteresting tedious'
#good_review = ''
#bad_review = ''

In [8]:
'''nan_array = list(pd.isna(data['review']))
for i in range(len(nan_array)):
    if nan_array[i]:
        if data.quality[i] == 1:
            data.review[i] = good_review
        else:
            data.review[i] = bad_review'''

"nan_array = list(pd.isna(data['review']))\nfor i in range(len(nan_array)):\n    if nan_array[i]:\n        if data.quality[i] == 1:\n            data.review[i] = good_review\n        else:\n            data.review[i] = bad_review"

In [9]:
for i in range(data.shape[0]):
    if data.raiting[i] >=7:
        data.quality[i] = 1
    else:
        data.quality[i] = 0

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [10]:
data.index[6]

6

In [11]:
arr = []
nan_array = list(pd.isna(data['review']))
for i in range(data.shape[0]):
    if data.raiting[i] < 7 and data.raiting[i] > 6:
        arr.append(i)
    if i not in arr and nan_array[i]:
        arr.append(i)

In [12]:
data = data.drop(arr)

In [13]:
data

,name,review,raiting,quality
0,Toy Story (1995),as the favourite characters for adults.Overall...,8.3,1
3,Waiting to Exhale (1995),"unfeeling creatures. So basically, I feel that...",5.8,0
4,Father of the Bride Part II (1995),I don't think George Banks could possibly have...,6.0,0
5,Heat (1995),want to lead. Pacino's cop is less easy to sym...,8.2,1
7,Tom and Huck (1995),scenes. The graveyard scene is well done. Inju...,5.6,0
8,Sudden Death (1995),"the gunplay and the action during the game), a...",5.7,0
9,GoldenEye (1995),"produced by Eon Productions, and the first to ...",7.2,1
11,Dracula: Dead and Loving It (1995),engaged couple who after 10 years have suddenl...,5.8,0
12,Balto (1995),of the movie. It especially sounds great when ...,7.1,1
13,Nixon (1995),ready for the day when they turned on him. It...,7.1,1


In [14]:
len(set(data.name))

2865

In [15]:
len(data.name)

2865

In [16]:
#data = data.dropna()

In [17]:
data.shape

(2865, 4)

In [18]:
#for i in range(data.shape[0]):
    #print(i)
    #data['review'][i] = 0 if data['raiting'][i] < 7 else 1

In [19]:
lemmatizer = WordNetLemmatizer()

In [20]:
import string
exclude = set(string.punctuation)

In [21]:
data['review'] = data['review'].apply(lambda x:
                                      "".join(i for i in x if i not in exclude))

In [22]:
data.to_csv('film_reviews_4493_changed.csv', sep='\t', index=False)

In [15]:
data['review'] = data['review'].apply(lambda x:
                                      " ".join(lemmatizer.lemmatize(i) for i in x.split()))

In [24]:
title = "sfsfd sdf sf sfd"

In [25]:
" ".join(lemmatizer.lemmatize(i) for i in title.split())


'sfsfd sdf sf sfd'

In [35]:
vectorizer = CountVectorizer().fit(data['review'])
features = vectorizer.transform(data['review'])

In [36]:
clf = LogisticRegression()
clf.fit(features[:data.shape[0]-400], data.quality[:data.shape[0]-400])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [37]:
pd.set_option('display.max_rows',753)

In [38]:
data

,name,review,raiting,quality
0,Toy Story (1995),a the favourite character for adultsOverall th...,8.3,1
3,Waiting to Exhale (1995),unfeeling creature So basically I feel that th...,5.8,0
4,Father of the Bride Part II (1995),I dont think George Banks could possibly have ...,6.0,0
5,Heat (1995),want to lead Pacinos cop is le easy to sympath...,8.2,1
7,Tom and Huck (1995),scene The graveyard scene is well done Injun J...,5.6,0
8,Sudden Death (1995),the gunplay and the action during the game and...,5.7,0
9,GoldenEye (1995),produced by Eon Productions and the first to s...,7.2,1
11,Dracula: Dead and Loving It (1995),engaged couple who after 10 year have suddenly...,5.8,0
12,Balto (1995),of the movie It especially sound great when yo...,7.1,1
13,Nixon (1995),ready for the day when they turned on him It g...,7.1,1


In [39]:
data.shape

(2865, 4)

In [ ]:
review = 

In [40]:
#МЕТРИКА

In [41]:
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

In [42]:
y_true = np.array(list(data.quality[data.shape[0]-399:]))#метки
y_pred = np.array(list(clf.predict(features[data.shape[0]-399:])))#предсказания
t = 0.4

In [43]:
accuracy_score(y_true, y_pred, t)

0.8395989974937343

In [44]:
recall_score(y_true, y_pred, t)

0.8609865470852018

In [45]:
precision_score(y_true, y_pred, t)

0.8533333333333334

In [46]:
diction_ = [ (k, v) for k, v in vectorizer.vocabulary_.items()]
diction_.sort(key=lambda x: x[1])
diction_ = pd.DataFrame(diction_, columns=['Feature', 'Position'])
diction_.head()

,Feature,Position
0,007,0
1,00s,1
2,010,2
3,02,3
4,06,4


In [47]:
diction_['Weight'] = clf.coef_[0]

In [48]:
diction_.sort_values('Weight')

,Feature,Position,Weight
3265,bad,3265,-0.994799
41258,woman,41258,-0.722240
41437,worst,41437,-0.699546
18429,if,18429,-0.682843
14312,flick,14312,-0.665881
20580,kid,20580,-0.638956
18375,id,18375,-0.631326
8994,cute,8994,-0.622533
41439,worth,41439,-0.587270
6573,check,6573,-0.564665


In [26]:
s= 'sd&f'

In [27]:
s.replace('&', ' ')

'sd f'